In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from os.path import exists

sys.path.append('../..')

In [3]:
import pylab as plt
import pandas as pd
import numpy as np
from loguru import logger
import seaborn as sns
import copy

In [4]:
from vimms.Common import POSITIVE, set_log_level_warning, load_obj, save_obj
from vimms.ChemicalSamplers import UniformRTAndIntensitySampler, GaussianChromatogramSampler, UniformMZFormulaSampler, \
    MZMLFormulaSampler, MZMLRTandIntensitySampler, MZMLChromatogramSampler
from vimms.Noise import UniformSpikeNoise
from vimms.Evaluation import evaluate_real
from vimms.Chemicals import ChemicalMixtureFromMZML
from vimms.Roi import RoiBuilderParams, SmartRoiParams

from mass_spec_utils.data_import.mzmine import load_picked_boxes

from vimms_gym.env import DDAEnv
from vimms_gym.chemicals import generate_chemicals
from vimms_gym.evaluation import evaluate, run_method
from vimms_gym.common import METHOD_RANDOM, METHOD_FULLSCAN, METHOD_TOPN, METHOD_DQN, \
    METHOD_DQN_COV, METHOD_DQN_INT, METHOD_DQN_MID, METHOD_PPO, METHOD_PPO_RECURRENT
from vimms_gym.experiments import preset_qcb_large

# 1. Parameters

In [5]:
env_alpha = 0.25
env_beta = 0.50
extract = True
params, max_peaks = preset_qcb_large(None, alpha=env_alpha, beta=env_beta, extract_chromatograms=extract)
params, max_peaks

2023-03-07 17:19:31.708 | INFO     | vimms_gym.experiments:get_samplers:295 - Loaded /Users/joewandy/Work/git/vimms-gym/pickles/samplers_QCB_large_extracted.p


({'chemical_creator': {'mz_range': (70, 1000),
   'rt_range': (0, 1440),
   'intensity_range': (10000.0, 1e+20),
   'n_chemicals': (2000, 5000),
   'mz_sampler': <vimms.ChemicalSamplers.MZMLFormulaSampler at 0x7feb8ad2df10>,
   'ri_sampler': <vimms.ChemicalSamplers.MZMLRTandIntensitySampler at 0x7feb7a5d9f10>,
   'cr_sampler': <vimms.ChemicalSamplers.MZMLChromatogramSampler at 0x7feb7a6387f0>},
  'noise': {'enable_spike_noise': True,
   'noise_density': 0.1,
   'noise_max_val': 1000.0,
   'mz_range': (70, 1000)},
  'env': {'ionisation_mode': 'Positive',
   'rt_range': (0, 1440),
   'isolation_window': 0.7,
   'use_dew': False,
   'mz_tol': 10,
   'rt_tol': 5,
   'min_ms1_intensity': 5000,
   'alpha': 0.25,
   'beta': 0.5}},
 200)

In [6]:
max_peaks = 200
out_dir = 'optimise_baselines'

In [7]:
n_eval_episodes = 1

# 2. Evaluation

#### Generate some chemical sets

In [8]:
fname = 'QCB_chems_large.p'
found = exists(fname)
if found:
    chem_list = load_obj(fname)
    for chems in chem_list:
        print(len(chems))

2599
4191
4880
3447
3722
2383
4001
3885
3295
2598
2013
4096
2826
4601
4042
4292
4929
3450
4241
4591
2400
4777
2012
4104
2521
2364
4850
4326
4153
3219


In [9]:
chem_list = [chem_list[0]]

#### Compare to Top-10

In [10]:
set_log_level_warning()

1

In [11]:
env_name = 'DDAEnv'
method = 'topN'
intensity_threshold = 0.5

In [12]:
rt_tols = [2, 5, 15, 30, 60, 120, 240, 300]
Ns = [1, 2, 5, 10, 15, 20, 25]

In [13]:
min_ms1_intensity = params['env']['min_ms1_intensity']
horizon = 4

In [14]:
topN_res = {}
for topN_rt_tol in rt_tols:
    for topN_N in Ns:

        copy_params = copy.deepcopy(params)            
        custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
        }    

        model = None
        if method == METHOD_TOPN:
            N = topN_N
            effective_rt_tol = topN_rt_tol
            copy_params = dict(params)
            copy_params['env']['use_dew'] = True
            copy_params['env']['rt_tol'] = effective_rt_tol                        

        banner = 'method = %s max_peaks = %d N = %d rt_tol = %d' % (method, max_peaks, N, copy_params['env']['rt_tol'])
        print(banner)
        print()            

        episodic_results = run_method(env_name, copy_params, max_peaks, chem_list, method, out_dir, 
                                      N=N, min_ms1_intensity=min_ms1_intensity, model=model,
                                      print_eval=True, print_reward=False, intensity_threshold=intensity_threshold,
                                      mzml_prefix=method, horizon=horizon, write_mzML=False)
        eval_results = [er.eval_res for er in episodic_results]

        key = (topN_N, topN_rt_tol)
        topN_res[key] = eval_results
        print()    

method = topN max_peaks = 200 N = 1 rt_tol = 2

{'coverage_prop': '0.279', 'intensity_prop': '0.150', 'ms1ms2_ratio': '1.040', 'efficiency': '0.311', 'TP': '404', 'FP': '322', 'FN': '1873', 'precision': '0.556', 'recall': '0.177', 'f1': '0.269', 'total_rewards': 417.2369200938458, 'invalid_action_count': 0, 'num_ms1_scans': 2431, 'num_ms2_scans': 2338}

method = topN max_peaks = 200 N = 2 rt_tol = 2

{'coverage_prop': '0.320', 'intensity_prop': '0.184', 'ms1ms2_ratio': '0.540', 'efficiency': '0.240', 'TP': '495', 'FP': '336', 'FN': '1768', 'precision': '0.596', 'recall': '0.219', 'f1': '0.320', 'total_rewards': 599.6288116950238, 'invalid_action_count': 0, 'num_ms1_scans': 1870, 'num_ms2_scans': 3461}

method = topN max_peaks = 200 N = 5 rt_tol = 2

{'coverage_prop': '0.392', 'intensity_prop': '0.236', 'ms1ms2_ratio': '0.240', 'efficiency': '0.210', 'TP': '636', 'FP': '384', 'FN': '1579', 'precision': '0.624', 'recall': '0.287', 'f1': '0.393', 'total_rewards': 828.2891161053655, 'inval

In [15]:
method_eval_results = {
    method: topN_res
}

#### Test classic controllers in ViMMS

In [16]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import TopNController, TopN_SmartRoiController, WeightedDEWController
from vimms.Environment import Environment

In [17]:
params

{'chemical_creator': {'mz_range': (70, 1000),
  'rt_range': (0, 1440),
  'intensity_range': (10000.0, 1e+20),
  'n_chemicals': (2000, 5000),
  'mz_sampler': <vimms.ChemicalSamplers.MZMLFormulaSampler at 0x7feb8ad2df10>,
  'ri_sampler': <vimms.ChemicalSamplers.MZMLRTandIntensitySampler at 0x7feb7a5d9f10>,
  'cr_sampler': <vimms.ChemicalSamplers.MZMLChromatogramSampler at 0x7feb7a6387f0>},
 'noise': {'enable_spike_noise': True,
  'noise_density': 0.1,
  'noise_max_val': 1000.0,
  'mz_range': (70, 1000)},
 'env': {'ionisation_mode': 'Positive',
  'rt_range': (0, 1440),
  'isolation_window': 0.7,
  'use_dew': True,
  'mz_tol': 10,
  'rt_tol': 300,
  'min_ms1_intensity': 5000,
  'alpha': 0.25,
  'beta': 0.5}}

In [18]:
enable_spike_noise = params['noise']['enable_spike_noise']
ionisation_mode = params['env']['ionisation_mode']
isolation_window = params['env']['isolation_window']
mz_tol = params['env']['mz_tol']
rt_range = params['chemical_creator']['rt_range']

In [19]:
spike_noise = None
if enable_spike_noise:
    noise_params = params['noise']
    noise_density = noise_params['noise_density']
    noise_max_val = noise_params['noise_max_val']
    noise_min_mz = noise_params['mz_range'][0]
    noise_max_mz = noise_params['mz_range'][1]
    spike_noise = UniformSpikeNoise(noise_density, noise_max_val, min_mz=noise_min_mz,
                                    max_mz=noise_max_mz)

Run Top-N Controller

In [20]:
rt_range

(0, 1440)

In [21]:
method = 'TopN_Controller'
print('method = %s' % method)
print()

chems = chem_list[0]
res = {}
for rt_tol in rt_tols:
    for N in Ns:

        effective_rt_tol = rt_tol
        mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)
        controller = TopNController(ionisation_mode, N, isolation_window, mz_tol, rt_tol,
                                    min_ms1_intensity)
        env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False)
        env.run()
        eval_res = evaluate(env, intensity_threshold)
        key = (N, rt_tol)
        print(N, rt_tol, eval_res)
        res[key] = eval_res

method_eval_results[method] = res

method = TopN_Controller

1 2 {'coverage_prop': '0.279', 'intensity_prop': '0.150', 'ms1ms2_ratio': '1.040', 'efficiency': '0.311', 'TP': '404', 'FP': '322', 'FN': '1873', 'precision': '0.556', 'recall': '0.177', 'f1': '0.269'}
2 2 {'coverage_prop': '0.339', 'intensity_prop': '0.200', 'ms1ms2_ratio': '0.542', 'efficiency': '0.255', 'TP': '549', 'FP': '331', 'FN': '1719', 'precision': '0.624', 'recall': '0.242', 'f1': '0.349'}
5 2 {'coverage_prop': '0.466', 'intensity_prop': '0.292', 'ms1ms2_ratio': '0.243', 'efficiency': '0.250', 'TP': '809', 'FP': '402', 'FN': '1388', 'precision': '0.668', 'recall': '0.368', 'f1': '0.475'}
10 2 {'coverage_prop': '0.474', 'intensity_prop': '0.300', 'ms1ms2_ratio': '0.144', 'efficiency': '0.220', 'TP': '841', 'FP': '392', 'FN': '1366', 'precision': '0.682', 'recall': '0.381', 'f1': '0.489'}
15 2 {'coverage_prop': '0.586', 'intensity_prop': '0.385', 'ms1ms2_ratio': '0.111', 'efficiency': '0.258', 'TP': '1095', 'FP': '427', 'FN': '1077', 'precision': '0.7

Run SmartROI Controller

In [24]:
alphas = [2, 3, 5, 10, 1E3, 1E6]
betas = [0, 0.1, 0.5, 1, 5]
initial_length_seconds_list = [0, 2, 5]
smartroi_N = 15
smartroi_dew = 15

In [25]:
method = 'SmartROI_Controller'
print('method = %s' % method)
print()

chems = chem_list[0]
res = {}
for alpha in alphas:
    for beta in betas:
        for ils in initial_length_seconds_list:

            mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)

            roi_params = RoiBuilderParams(min_roi_intensity=0, min_roi_length=0)    
            smartroi_params = SmartRoiParams(intensity_increase_factor=alpha, drop_perc=beta/100.0, dew=smartroi_dew, initial_length_seconds=ils)
            controller = TopN_SmartRoiController(ionisation_mode, isolation_window, smartroi_N, mz_tol, smartroi_dew,
                                        min_ms1_intensity, roi_params, smartroi_params)

            env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False)
            env.run()
            eval_res = evaluate(env, intensity_threshold)
            key = (alpha, beta, ils)
            print(alpha, beta, ils, eval_res)
            res[key] = eval_res

method_eval_results[method] = res

method = SmartROI_Controller

2 0 0 {'coverage_prop': '0.943', 'intensity_prop': '0.600', 'ms1ms2_ratio': '0.669', 'efficiency': '0.796', 'TP': '1723', 'FP': '728', 'FN': '148', 'precision': '0.703', 'recall': '0.921', 'f1': '0.797'}
2 0 2 {'coverage_prop': '0.938', 'intensity_prop': '0.601', 'ms1ms2_ratio': '0.693', 'efficiency': '0.808', 'TP': '1725', 'FP': '714', 'FN': '160', 'precision': '0.707', 'recall': '0.915', 'f1': '0.798'}
2 0 5 {'coverage_prop': '0.940', 'intensity_prop': '0.616', 'ms1ms2_ratio': '0.724', 'efficiency': '0.830', 'TP': '1778', 'FP': '664', 'FN': '157', 'precision': '0.728', 'recall': '0.919', 'f1': '0.812'}
2 0.1 0 {'coverage_prop': '0.943', 'intensity_prop': '0.602', 'ms1ms2_ratio': '0.665', 'efficiency': '0.793', 'TP': '1727', 'FP': '725', 'FN': '147', 'precision': '0.704', 'recall': '0.922', 'f1': '0.798'}
2 0.1 2 {'coverage_prop': '0.939', 'intensity_prop': '0.603', 'ms1ms2_ratio': '0.692', 'efficiency': '0.808', 'TP': '1721', 'FP': '719', 'FN': '159', 'p

Run WeightedDEW Controller

In [26]:
t0s = [1, 3, 10, 15, 30, 60]
t1s = [15, 60, 120, 240, 360, 3600]
weighteddew_N = 15

In [27]:
method = 'WeightedDEW_Controller'
print('method = %s' % method)
print()

chems = chem_list[0]
res = {}
for t0 in t0s:
    for t1 in t1s:

        if t0 > t1:
            print('Invalid combination')
            continue
        
        mass_spec = IndependentMassSpectrometer(ionisation_mode, chems, spike_noise=spike_noise)
        
        controller = WeightedDEWController(ionisation_mode, weighteddew_N, isolation_window, mz_tol, t1,
                                    min_ms1_intensity, exclusion_t_0=t0)
        
        env = Environment(mass_spec, controller, rt_range[0], rt_range[1], progress_bar=False)
        env.run()
        eval_res = evaluate(env, intensity_threshold)
        key = (t0, t1)
        print(t0, t1, eval_res)
        res[key] = eval_res
        
method_eval_results[method] = res

method = WeightedDEW_Controller

1 15 {'coverage_prop': '0.786', 'intensity_prop': '0.554', 'ms1ms2_ratio': '0.097', 'efficiency': '0.339', 'TP': '1650', 'FP': '393', 'FN': '556', 'precision': '0.808', 'recall': '0.748', 'f1': '0.777'}
1 60 {'coverage_prop': '0.896', 'intensity_prop': '0.641', 'ms1ms2_ratio': '0.097', 'efficiency': '0.386', 'TP': '1939', 'FP': '389', 'FN': '271', 'precision': '0.833', 'recall': '0.877', 'f1': '0.855'}
1 120 {'coverage_prop': '0.923', 'intensity_prop': '0.659', 'ms1ms2_ratio': '0.097', 'efficiency': '0.398', 'TP': '2006', 'FP': '393', 'FN': '200', 'precision': '0.836', 'recall': '0.909', 'f1': '0.871'}
1 240 {'coverage_prop': '0.939', 'intensity_prop': '0.661', 'ms1ms2_ratio': '0.097', 'efficiency': '0.405', 'TP': '1995', 'FP': '446', 'FN': '158', 'precision': '0.817', 'recall': '0.927', 'f1': '0.869'}
1 360 {'coverage_prop': '0.945', 'intensity_prop': '0.663', 'ms1ms2_ratio': '0.097', 'efficiency': '0.408', 'TP': '1997', 'FP': '460', 'FN': '142', 'prec